<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/test_with_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network applied in Aggregate Computing
In this notebook, I tried to apply Recurrent Neural Networks (RNN) in the context of Aggregate Computing (AC).

## Model

Usually, RNN are trained in indipendent sequences. In case of Aggregate Computing, the temporal sequence are correlated with each other following a neighbourhood policy. 

The key idea here is:
- express the system as a graph. Offline we can imagine to access to the entire system
- in each time step, we aggregate data from the neighbours and then we use RNN to compute the right output

# Imports

In [1]:
import tensorflow as tf
import tensorboard
from datetime import datetime
from numpy.random import seed
%load_ext tensorboard

# Simple Graph

In this model, I model graph as:
$G = (N, E)$

Where, $N$ contains a feature vector and an output vector.

$E$ is expressed as adjacency matrix, so: $E_{i,j} = 1$ iff $i$ is neighbour of $j$.

Input is a matrix contains the concatenation of feature vector and output vector, something like: $ I(n) = <f(n), o(n)>$.
Where $<a, b, c>$ means the column-wise fector concatenation. 
For instance, giving a vector $a = [1, 2]$ and $b = [2, 3]$, $<a,b> = [1, 2, 2, 3]$.

The neighbour aggregation data can be computed leveraging matrix multiplication --- so increasing traning performance.
$$ reduction(E * F) $$
Where F contains all node features.
$reduction$ is a multiset function (e.g. summation, mean,...).

Pay attention, this model can be easly used in a decentralized situation.
Indeed, the data aggregation will be done without global adjency matrix but only retrieving neighbour data. The dense layer already works locally.

In [29]:
n = 50000
feature = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                    ], dtype=tf.float32)
output = tf.constant([
                     [0], 
                     [n],
                     [n],
                     [n],
                    ], dtype=tf.float32)

input = tf.concat([feature, output], axis = 1)
input = input[:,tf.newaxis, :]

neigh = tf.constant([
                     [n, 1, n, n], 
                     [1, n, 1, n],
                     [n, 1, n, 1],
                     [n, n, 1, n],
                    ], dtype=tf.float32)

ground = tf.constant([
                     [1, 0], 
                     [0, 1],
                     [0, 2],
                     [0, 3],
                    ], dtype=tf.float32)

# Forward Pass

Taking the output of previous step, adjecency matrix, and feature vector, this function compute the value of the next evaluation.
So
1. compute neighbourhood feature via aggregation
2. concat neighbourhood feature with local feature and previous output
3. perform a forward pass of a neural network.


In [47]:
def forward(result, neigh, feature, eval_logic, log_enable=False):
  if(log_enable):
    print("Input = ", result)
  reshape = tf.reshape(result[:,:, 1], feature.shape[0])
  neigh_evaluation = tf.reduce_min(tf.multiply(neigh, reshape), 1) ## pass reduction strategy
  input_network = tf.concat([result[:,:, 0], neigh_evaluation[:, tf.newaxis]], 1)
  if(log_enable):
    print("Neighbour Aggregation = ", input_network[:, tf.newaxis])
  result = eval_logic(input_network[:, tf.newaxis])
  result = tf.concat([feature, result], axis = 1)
  if(log_enable):
    print("Output = ", result)
  return tf.expand_dims(result, [1])

# Model creation
Create a sequential model given layers and the input shape

In [24]:
def create_model(input_shape, layers):
  input_layer = tf.keras.layers.InputLayer(input_shape=input.shape[1:], batch_size=input_shape[0])
  layers.insert(0, input_layer)
  return tf.keras.Sequential(layers)

In [25]:
def instantiate_layers():
  return [
      tf.keras.layers.LSTM(units = 4, activation='relu', return_sequences=True, stateful=True),
      tf.keras.layers.LSTM(units = 1, activation='relu', return_sequences=False, stateful=True, bias_initializer='ones'),
  ]
model = create_model(input.shape, instantiate_layers())

# Train function

In [30]:
## TODO pass input and network
def train(model, data, iteration, stabilise_in, stabilisation_check, loss, optimizer, each=100):
  input, neigh, feature, ground = data
  for j in range(iteration):
    with tf.GradientTape() as tape:
      result = input
      to_backprop = 0
      for i in range(stabilise_in):
        result = forward(result, neigh, feature, model)
      for i in range(stabilisation_check):
        result = forward(result, neigh, feature, model)
        to_backprop += 1 / stabilisation_check * loss(ground, result[:, 0, :])
      model.reset_states()
      gradient = tape.gradient(to_backprop, model.weights)
      optimizer.apply_gradients(zip(gradient, model.weights))
    if(j % each == 0):
      print("Epoch ", j ,"Loss = ", tf.reduce_sum(to_backprop))  


## Train loop
Here I want only to find a function that overfits, so it solve this specific problem.


In [46]:
#seed(42)
#tf.random.set_seed(42)
model = create_model(input.shape, instantiate_layers()) ## comment to avoid the recomputation of weights
iteration = 10000
stabilise_in = 4
stabilisation_check = 5

loss = tf.losses.mse
optimizer = tf.optimizers.Adam()
data = (input, neigh, feature, ground)
train(model, data, iteration, stabilise_in, stabilisation_check, loss, optimizer, 50)

result = input
for i in range(20):
  result = forward(result, neigh, feature, model)
print(result)
model.reset_states()

ValueError: ignored

# Check result, generalisation
In this part, I try to use the same network in another graph, to see if it can be used in different graphs.

In [36]:
n = 50000
feature_validation = tf.constant([
                     [0], 
                     [0],
                     [0],
                     [0],
                     [0]
                    ], dtype=tf.float32)
output_validation = tf.constant([
                     [0], 
                     [n],
                     [n],
                     [n],
                     [n]
                    ], dtype=tf.float32)
input_validation = tf.concat([feature_validation, output_validation], axis = 1)
input_validation = input_validation[:,tf.newaxis, :]
neigh_validation = tf.constant([
                     [n, 1, n, n, n], 
                     [1, n, 1, n, n],
                     [n, 1, n, 1, n],
                     [n, n, 1, n, 1],
                     [n, n, n, 1, n],
                    ], dtype=tf.float32)

change_model = create_model(input_validation.shape, instantiate_layers())
change_model.set_weights(model.get_weights())

result_validation = input_validation
model.reset_states()
for i in range(20):
  result_validation = forward(result_validation, neigh_validation, feature_validation, change_model, False)
print(result_validation)

tf.Tensor(
[[[0.0000000e+00 1.2036740e+00]]

 [[0.0000000e+00 1.2035980e+00]]

 [[1.0000000e+00 1.1088027e-27]]

 [[0.0000000e+00 1.2035980e+00]]

 [[0.0000000e+00 1.2036740e+00]]], shape=(5, 1, 2), dtype=float32)
